In [2]:
# Exercise: 9-1
# What constraints are required to correctly parse word sequences like I am happy and she is happy but not
# *you is happy or *they am happy? Implement two solutions for the present tense paradigm of the verb be in English,
# first taking Grammar Example 9-8 as your starting point, and then taking Grammar Example 9-22 as the starting point.

import nltk

grammar = nltk.CFG.fromstring("""
S -> NP_SG_1 VP_SG_1
S -> NP_SG_2 VP_SG_2
S -> NP_SG_3 VP_SG_3
S -> NP_PL VP_PL
NP_SG_1 -> N_SG_1
NP_SG_2 -> N_SG_2
NP_SG_3 -> N_SG_3
NP_PL -> N_PL_1 | N_PL_2 | N_PL_3
VP_SG_1 -> V_SG_1 ADJ
VP_SG_2 -> V_SG_2 ADJ
VP_SG_3 -> V_SG_3 ADJ
VP_PL -> V_PL ADJ
N_SG_1 -> "I"
N_SG_2 -> "you"
N_SG_3 -> "he" | "she" | "it"
N_PL_1 -> "we"
N_PL_2 -> "you"
N_PL_3 -> "they"
V_SG_1 -> "am"
V_SG_2 -> "are"
V_SG_3 -> "is"
V_PL   -> "are"
ADJ -> "happy"
""")

sent = "I am happy".split()
parser = nltk.ChartParser(grammar)
for tree in parser.parse(sent):
    print(tree)

(S (NP_SG_1 (N_SG_1 I)) (VP_SG_1 (V_SG_1 am) (ADJ happy)))


In [3]:
sent = "she is happy".split()
for tree in parser.parse(sent):
    print(tree)

(S (NP_SG_3 (N_SG_3 she)) (VP_SG_3 (V_SG_3 is) (ADJ happy)))


In [4]:
sent = "you is happy".split()
for tree in parser.parse(sent):
    print(tree)

In [5]:
sent = "they am happy".split()
for tree in parser.parse(sent):
    print(tree)

In [6]:
from nltk import grammar, parse

g = """
% start S
S -> NP[AGR = ?n] VP[AGR = ?n]
NP[AGR = ?n] -> N[AGR = ?n]
VP[AGR = ?n] -> V[AGR = ?n] ADJ
N[AGR = [PER = 1, NUM = sg]] -> "I"
N[AGR = [PER = 2, NUM = sg]] -> "you"
N[AGR = [PER = 3, NUM = sg]] -> "he" | "she" | "it"
N[AGR = [PER = 1, NUM = pl]] -> "we"
N[AGR = [PER = 2, NUM = pl]] -> "you"
N[AGR = [PER = 3, NUM = pl]]-> "they"
V[AGR = [PER = 1, NUM = sg]] -> "am"
V[AGR = [PER = 2, NUM = sg]] -> "are"
V[AGR = [PER = 3, NUM = sg]] -> "is"
V[AGR = [PER = ?n, NUM = pl]]  -> "are"
ADJ -> "happy"
"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram)

sent = "I am happy".split()

for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[AGR=[NUM='sg', PER=1]] (N[AGR=[NUM='sg', PER=1]] I))
  (VP[AGR=[NUM='sg', PER=1]]
    (V[AGR=[NUM='sg', PER=1]] am)
    (ADJ[] happy)))


In [7]:
sent = "she is happy".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[AGR=[NUM='sg', PER=3]] (N[AGR=[NUM='sg', PER=3]] she))
  (VP[AGR=[NUM='sg', PER=3]]
    (V[AGR=[NUM='sg', PER=3]] is)
    (ADJ[] happy)))


In [8]:
sent = "they are happy".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[AGR=[NUM='pl', PER=3]] (N[AGR=[NUM='pl', PER=3]] they))
  (VP[AGR=[NUM='pl', PER=?n2]]
    (V[AGR=[NUM='pl', PER=?n]] are)
    (ADJ[] happy)))


In [9]:
sent = "you is happy".split()
for tree in parser.parse(sent):
    print(tree)

In [10]:
sent = "they am happy".split()
for tree in parser.parse(sent):
    print(tree)

In [11]:
# Exercise: 9-2
# Develop a variant of grammar in Example 9-17 that uses a feature COUNT to make the distinctions shown here:
# Example 9-60.
# a. The boy sings.
# b. *Boy sings.

# Example 9-61.
# a. The boys sing.
# b. Boys sing.

# Example 9-62.
# a. The water is precious.
# b. Water is precious.

g = """
% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n, Count = ?c] -> Det[NUM=?n] N[NUM=?n, Count = ?c]
NP[NUM=pl, Count = ?c] -> N[NUM=pl, Count = ?c]
NP[NUM=?n, Count = False] -> N[NUM=?n, Count = False]
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> ambiV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> Cop[TENSE=?t, NUM=?n] ADJ
# ###################
# Lexical Productions
# ###################
Det -> 'The' | 'the'
N[NUM=sg, Count = True] -> 'boy' | 'Boy'
N[NUM=sg, Count = False] -> 'water' | 'Water'
N[NUM=pl, Count = True] -> 'Boys' | 'boys'
ambiV[TENSE=pres,  NUM=sg] -> 'sings'
ambiV[TENSE=pres,  NUM=pl] -> 'sing'
Cop[TENSE=pres,  NUM=sg] -> 'is'
Cop[TENSE=pres,  NUM=pl] -> 'are'
ADJ -> 'precious'
"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram)

In [12]:
sent = "the boy sings".split()

for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[+Count, NUM='sg'] (Det[] the) (N[+Count, NUM='sg'] boy))
  (VP[NUM='sg', TENSE='pres'] (ambiV[NUM='sg', TENSE='pres'] sings)))


In [13]:
sent = "Boy sings".split()
for tree in parser.parse(sent):
    print(tree)

In [14]:
sent = "The boys sing".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[+Count, NUM='pl'] (Det[] The) (N[+Count, NUM='pl'] boys))
  (VP[NUM='pl', TENSE='pres'] (ambiV[NUM='pl', TENSE='pres'] sing)))


In [15]:
sent = "Boys sing".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[+Count, NUM='pl'] (N[+Count, NUM='pl'] Boys))
  (VP[NUM='pl', TENSE='pres'] (ambiV[NUM='pl', TENSE='pres'] sing)))


In [16]:
sent = "The water is precious".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[-Count, NUM='sg'] (Det[] The) (N[-Count, NUM='sg'] water))
  (VP[NUM='sg', TENSE='pres']
    (Cop[NUM='sg', TENSE='pres'] is)
    (ADJ[] precious)))


In [17]:
sent = "Water is precious".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[-Count, NUM='sg'] (N[-Count, NUM='sg'] Water))
  (VP[NUM='sg', TENSE='pres']
    (Cop[NUM='sg', TENSE='pres'] is)
    (ADJ[] precious)))


In [18]:
# Exercise: 9-3
# Write a function subsumes() that holds of two feature structures fs1 and fs2 just in case fs1 subsumes fs2.

def subsumes(f1, f2):
    """
    Checks two features structures to see if one subsumes the other.

    Arguments:
    f1, f2: two nltk.FeatStruct objects

    Returns:
    0:   Neither object subsumes the other
    1:   First object subsumes the second
    2:   Second object subsumes the first
    """

    for f in [f1, f2]:
        assert isinstance(f, nltk.FeatStruct), \
            "Argument must be an nltk.FeatStruct"

    if f1 == f1.unify(f2):
        return 1
    elif f2 == f2.unify(f1):
        return 2
    else:
        return 0

In [19]:
fs1 = nltk.FeatStruct(NAME = "Jack Torrance", ROOM = '237',
                      LODGING = 'Overlook Hotel')
fs2 = nltk.FeatStruct(ROOM = '237')

In [20]:
subsumes(fs1, fs2)

1

In [21]:
fs1 = nltk.FeatStruct(ROOM = '237')
fs2 = nltk.FeatStruct(NAME = "Jack Torrance", ROOM = '237',
                      LODGING = 'Overlook Hotel')

In [22]:
subsumes(fs1, fs2)

2

In [23]:
fs1 = nltk.FeatStruct(ROOM = '237')
fs2 = nltk.FeatStruct(NAME = "Jack Torrance", LODGING = 'Overlook Hotel')

In [24]:
subsumes(fs1, fs2)

0

In [25]:
# Exercise: 9-5
# Modify the German grammar in Example 9-59 to incorporate the treatment of subcategorization presented in
# Extending a Feature-Based Grammar.

g = """
% start S
# Grammar Productions
S -> NP[CASE=nom, AGR=?a] VP[AGR=?a]
NP[CASE=?c, AGR=?a] -> PRO[CASE=?c, AGR=?a]
NP[CASE=?c, AGR=?a] -> Det[CASE=?c, AGR=?a] N[CASE=?c, AGR=?a]
VP[AGR=?a] -> V[SUBCAT = intrans, AGR=?a]
VP[AGR=?a] -> V[SUBCAT = trans, OBJCASE=?c, AGR=?a] NP[CASE=?c]
# Lexical Productions
# Singular determiners
# masc
Det[CASE=nom, AGR=[GND=masc,PER=3,NUM=sg]] -> 'der'
Det[CASE=dat, AGR=[GND=masc,PER=3,NUM=sg]] -> 'dem'
Det[CASE=acc, AGR=[GND=masc,PER=3,NUM=sg]] -> 'den'
# fem
Det[CASE=nom, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die'
Det[CASE=dat, AGR=[GND=fem,PER=3,NUM=sg]] -> 'der'
Det[CASE=acc, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die'
# Plural determiners
Det[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'die'
Det[CASE=dat, AGR=[PER=3,NUM=pl]] -> 'den'
Det[CASE=acc, AGR=[PER=3,NUM=pl]] -> 'die'
# Nouns
N[AGR=[GND=masc,PER=3,NUM=sg]] -> 'Hund'
N[CASE=nom, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[CASE=dat, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunden'
N[CASE=acc, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[AGR=[GND=fem,PER=3,NUM=sg]] -> 'Katze'
N[AGR=[GND=fem,PER=3,NUM=pl]] -> 'Katzen'
# Pronouns
PRO[CASE=nom, AGR=[PER=1,NUM=sg]] -> 'ich'
PRO[CASE=acc, AGR=[PER=1,NUM=sg]] -> 'mich'
PRO[CASE=dat, AGR=[PER=1,NUM=sg]] -> 'mir'
PRO[CASE=nom, AGR=[PER=2,NUM=sg]] -> 'du'
PRO[CASE=nom, AGR=[PER=3,NUM=sg]] -> 'er' | 'sie' | 'es'
PRO[CASE=nom, AGR=[PER=1,NUM=pl]] -> 'wir'
PRO[CASE=acc, AGR=[PER=1,NUM=pl]] -> 'uns'
PRO[CASE=dat, AGR=[PER=1,NUM=pl]] -> 'uns'
PRO[CASE=nom, AGR=[PER=2,NUM=pl]] -> 'ihr'
PRO[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'sie'
# Verbs
V[SUBCAT = intrans, AGR=[NUM=sg,PER=1]] -> 'komme'
V[SUBCAT = intrans, AGR=[NUM=sg,PER=2]] -> 'kommst'
V[SUBCAT = intrans, AGR=[NUM=sg,PER=3]] -> 'kommt'
V[SUBCAT = intrans, AGR=[NUM=pl, PER=1]] -> 'kommen'
V[SUBCAT = intrans, AGR=[NUM=pl, PER=2]] -> 'kommt'
V[SUBCAT = intrans, AGR=[NUM=pl, PER=3]] -> 'kommen'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=sg,PER=1]] -> 'sehe' | 'mag'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=sg,PER=2]] -> 'siehst' | 'magst'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=sg,PER=3]] -> 'sieht' | 'mag'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=sg,PER=1]] -> 'folge' | 'helfe'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=sg,PER=2]] -> 'folgst' | 'hilfst'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=sg,PER=3]] -> 'folgt' | 'hilft'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=pl,PER=1]] -> 'sehen' | 'moegen'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=pl,PER=2]] -> 'sieht' | 'moegt'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=pl,PER=3]] -> 'sehen' | 'moegen'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=pl,PER=1]] -> 'folgen' | 'helfen'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=pl,PER=2]] -> 'folgt' | 'helft'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=pl,PER=3]] -> 'folgen' | 'helfen'
"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram)

In [26]:
Satz = "ich folge den Katzen".split()
for tree in parser.parse(Satz):
    print(tree)

(S[]
  (NP[AGR=[NUM='sg', PER=1], CASE='nom']
    (PRO[AGR=[NUM='sg', PER=1], CASE='nom'] ich))
  (VP[AGR=[NUM='sg', PER=1]]
    (V[AGR=[NUM='sg', PER=1], OBJCASE='dat', SUBCAT='trans'] folge)
    (NP[AGR=[GND='fem', NUM='pl', PER=3], CASE='dat']
      (Det[AGR=[NUM='pl', PER=3], CASE='dat'] den)
      (N[AGR=[GND='fem', NUM='pl', PER=3]] Katzen))))


In [27]:
Satz = "die Katze sieht den Hund".split()
for tree in parser.parse(Satz):
    print(tree)

(S[]
  (NP[AGR=[GND='fem', NUM='sg', PER=3], CASE='nom']
    (Det[AGR=[GND='fem', NUM='sg', PER=3], CASE='nom'] die)
    (N[AGR=[GND='fem', NUM='sg', PER=3]] Katze))
  (VP[AGR=[NUM='sg', PER=3]]
    (V[AGR=[NUM='sg', PER=3], OBJCASE='acc', SUBCAT='trans'] sieht)
    (NP[AGR=[GND='masc', NUM='sg', PER=3], CASE='acc']
      (Det[AGR=[GND='masc', NUM='sg', PER=3], CASE='acc'] den)
      (N[AGR=[GND='masc', NUM='sg', PER=3]] Hund))))


In [28]:
Satz = "die Katze sieht dem Hund".split()
for tree in parser.parse(Satz):
    print(tree)

In [29]:
Satz = "die Katze hilft dem Hund".split()
for tree in parser.parse(Satz):
    print(tree)

(S[]
  (NP[AGR=[GND='fem', NUM='sg', PER=3], CASE='nom']
    (Det[AGR=[GND='fem', NUM='sg', PER=3], CASE='nom'] die)
    (N[AGR=[GND='fem', NUM='sg', PER=3]] Katze))
  (VP[AGR=[NUM='sg', PER=3]]
    (V[AGR=[NUM='sg', PER=3], OBJCASE='dat', SUBCAT='trans'] hilft)
    (NP[AGR=[GND='masc', NUM='sg', PER=3], CASE='dat']
      (Det[AGR=[GND='masc', NUM='sg', PER=3], CASE='dat'] dem)
      (N[AGR=[GND='masc', NUM='sg', PER=3]] Hund))))


In [30]:
Satz = "die Katze hilft den Hund".split()
for tree in parser.parse(Satz):
    print(tree)

In [31]:
Satz = "ich folge den Katze".split()
for tree in parser.parse(Satz):
    print(tree)

In [32]:
# Exercise: 9-6
# Develop a feature-based grammar that will correctly describe the following Spanish noun phrases:

g = """
% start NP
# Grammar Productions

NP[AGR=?a] -> INDEF[AGR=?a] N[AGR=?a] ADJ[AGR=?a]

# Lexical Productions

# Singular determiners
INDEF[AGR=[GND=masc, NUM=sg]] -> 'un'
INDEF[AGR=[GND=masc, NUM=pl]] -> 'unos'
INDEF[AGR=[GND=fem, NUM=sg]] -> 'una'
INDEF[AGR=[GND=fem, NUM=pl]] -> 'unas'

# Nouns
N[AGR=[GND=masc, NUM=sg]] -> 'cuadro'
N[AGR=[GND=masc, NUM=pl]] -> 'cuadros'
N[AGR=[GND=fem, NUM=sg]] -> 'cortina'
N[AGR=[GND=fem, NUM=pl]] -> 'cortinas'

# Adjectives
ADJ[AGR=[GND=masc, NUM=sg]] -> 'hermoso'
ADJ[AGR=[GND=masc, NUM=pl]] -> 'hermosos'
ADJ[AGR=[GND=fem, NUM=sg]] -> 'hermosa'
ADJ[AGR=[GND=fem, NUM=pl]] -> 'hermosas'

"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram)

In [33]:
frase = "un cuadro hermoso".split()
for tree in parser.parse(frase):
    print(tree)

(NP[AGR=[GND='masc', NUM='sg']]
  (INDEF[AGR=[GND='masc', NUM='sg']] un)
  (N[AGR=[GND='masc', NUM='sg']] cuadro)
  (ADJ[AGR=[GND='masc', NUM='sg']] hermoso))


In [34]:
frase = "unos cuadros hermosos".split()
for tree in parser.parse(frase):
    print(tree)

(NP[AGR=[GND='masc', NUM='pl']]
  (INDEF[AGR=[GND='masc', NUM='pl']] unos)
  (N[AGR=[GND='masc', NUM='pl']] cuadros)
  (ADJ[AGR=[GND='masc', NUM='pl']] hermosos))


In [35]:
frase = "una cortina hermosa".split()
for tree in parser.parse(frase):
    print(tree)

(NP[AGR=[GND='fem', NUM='sg']]
  (INDEF[AGR=[GND='fem', NUM='sg']] una)
  (N[AGR=[GND='fem', NUM='sg']] cortina)
  (ADJ[AGR=[GND='fem', NUM='sg']] hermosa))


In [36]:
frase = "unas cortinas hermosas".split()
for tree in parser.parse(frase):
    print(tree)

(NP[AGR=[GND='fem', NUM='pl']]
  (INDEF[AGR=[GND='fem', NUM='pl']] unas)
  (N[AGR=[GND='fem', NUM='pl']] cortinas)
  (ADJ[AGR=[GND='fem', NUM='pl']] hermosas))


In [37]:
frase = "una cuadro hermosa".split()
for tree in parser.parse(frase):
    print(tree)

In [38]:
frase = "unos cuadro hermosos".split()
for tree in parser.parse(frase):
    print(tree)

In [39]:
# Exercise: 9-7
# Develop a wrapper for the earley_parser so that a trace is only printed if the input sequence fails to parse.

import nltk

def check_Earley(seq, parser):
    """
    Checks seq to determine if it would fail to parse.

    Args:

    seq:    The string to be checked in the parsed grammar
    parser: A pre-defined parser
    """
    phrase = seq.split()
    if not tree in parser.parse(phrase):
        print("This sequence fails to parse.")

In [40]:
check_Earley("un cuadro hermoso", parser)

This sequence fails to parse.


In [41]:
# Exercise: 9-8
# fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
# fs2 = nltk.FeatStruct("[B = [D = d]]")
# fs3 = nltk.FeatStruct("[B = [C = d]]")
# fs4 = nltk.FeatStruct("[A = (1)[B = b], C->(1)]")
# fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
# fs6 = nltk.FeatStruct("[A = [D = d]]")
# fs7 = nltk.FeatStruct("[A = [D = d], C = [F = [D = d]]]")
# fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
# fs9 = nltk.FeatStruct("[A = [B = b], C = [E = [G = e]]]")
# fs10 = nltk.FeatStruct("[A = (1)[B = b], C -> (1)]")
# Work out on paper what the result is of the following unifications. (Hint: you might find it useful to draw the graph structures.)

# a: fs1 and fs2
# b: fs1 and fs3
# c: fs4 and fs5
# d: fs5 and fs6
# e: fs5 and fs7
# f: fs8 and fs9
# g: fs8 and fs10

# Check your answers using NLTK.

fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
fs2 = nltk.FeatStruct("[B = [D = d]]")

print(fs1.unify(fs2))

[ A = ?x          ]
[                 ]
[ B = [ C = ?x  ] ]
[     [ D = 'd' ] ]


In [42]:
fs1 = nltk.FeatStruct("[A = ?x, B= [C = ?x]]")
fs3 = nltk.FeatStruct("[B = [C = d]]")

print(fs1.unify(fs3))

[ A = 'd'         ]
[                 ]
[ B = [ C = 'd' ] ]


In [43]:
fs4 = nltk.FeatStruct("[A = (1)[B = b], C->(1)]")
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")

print(fs4.unify(fs5))

[         [ B = 'b'  ] ]
[ A = (1) [ D = ?x   ] ]
[         [ E -> (1) ] ]
[         [ F = ?x   ] ]
[                      ]
[ C -> (1)             ]


In [44]:
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
fs6 = nltk.FeatStruct("[A = [D = d]]")

print(fs5.unify(fs6))

[ A = (1) [ D = 'd' ] ]
[                     ]
[ C = [ E -> (1) ]    ]
[     [ F = 'd'  ]    ]


In [45]:
fs5 = nltk.FeatStruct("[A = (1)[D = ?x], C = [E -> (1), F = ?x] ]")
fs7 = nltk.FeatStruct("[A = [D = d], C = [F = [D = d]]]")

print(fs5.unify(fs7))

None


In [46]:
fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
fs9 = nltk.FeatStruct("[A = [B = b], C = [E = [G = e]]]")

print(fs8.unify(fs9))

[         [ B = 'b' ] ]
[ A = (1) [ D = 'e' ] ]
[         [ G = 'e' ] ]
[                     ]
[ C = [ B = 'e'  ]    ]
[     [ E -> (1) ]    ]


In [47]:
fs8 = nltk.FeatStruct("[A = (1)[D = ?x, G = ?x], C = [B = ?x, E -> (1)] ]")
fs10 = nltk.FeatStruct("[A = (1)[B = b], C -> (1)]")

print(fs8.unify(fs10))

[         [ B = 'b'  ] ]
[ A = (1) [ D = 'b'  ] ]
[         [ E -> (1) ] ]
[         [ G = 'b'  ] ]
[                      ]
[ C -> (1)             ]


In [48]:
# Exercise: 9-11
# Extend the German grammar in Example 9-59 so that it can handle so-called verb-second structures
# like the following:

# Heute sieht der Hund die Katze.

g = """
% start S
# Grammar Productions
S -> NP[CASE=nom, AGR=?a] VP[AGR=?a]
S -> ADV VP[AGR=?a]

NP[CASE=?c, AGR=?a] -> PRO[CASE=?c, AGR=?a]
NP[CASE=?c, AGR=?a] -> Det[CASE=?c, AGR=?a] N[CASE=?c, AGR=?a]
VP[AGR=?a] -> V[SUBCAT = intrans, AGR=?a]
VP[AGR=?a] -> V[SUBCAT = trans, OBJCASE=?c, AGR=?a] NP[CASE=?c]

# New VP production
VP[AGR=?a] -> V[SUBCAT = trans, OBJCASE=?c, AGR=?a] NP[CASE=nom, AGR = ?a] NP[CASE=?c]

# Lexical Productions
# Singular determiners
# masc
Det[CASE=nom, AGR=[GND=masc,PER=3,NUM=sg]] -> 'der'
Det[CASE=dat, AGR=[GND=masc,PER=3,NUM=sg]] -> 'dem'
Det[CASE=acc, AGR=[GND=masc,PER=3,NUM=sg]] -> 'den'
# fem
Det[CASE=nom, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die'
Det[CASE=dat, AGR=[GND=fem,PER=3,NUM=sg]] -> 'der'
Det[CASE=acc, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die'
# Plural determiners
Det[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'die'
Det[CASE=dat, AGR=[PER=3,NUM=pl]] -> 'den'
Det[CASE=acc, AGR=[PER=3,NUM=pl]] -> 'die'
# Nouns
N[AGR=[GND=masc,PER=3,NUM=sg]] -> 'Hund'
N[CASE=nom, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[CASE=dat, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunden'
N[CASE=acc, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[AGR=[GND=fem,PER=3,NUM=sg]] -> 'Katze'
N[AGR=[GND=fem,PER=3,NUM=pl]] -> 'Katzen'
# Pronouns
PRO[CASE=nom, AGR=[PER=1,NUM=sg]] -> 'ich'
PRO[CASE=acc, AGR=[PER=1,NUM=sg]] -> 'mich'
PRO[CASE=dat, AGR=[PER=1,NUM=sg]] -> 'mir'
PRO[CASE=nom, AGR=[PER=2,NUM=sg]] -> 'du'
PRO[CASE=nom, AGR=[PER=3,NUM=sg]] -> 'er' | 'sie' | 'es'
PRO[CASE=nom, AGR=[PER=1,NUM=pl]] -> 'wir'
PRO[CASE=acc, AGR=[PER=1,NUM=pl]] -> 'uns'
PRO[CASE=dat, AGR=[PER=1,NUM=pl]] -> 'uns'
PRO[CASE=nom, AGR=[PER=2,NUM=pl]] -> 'ihr'
PRO[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'sie'
# Verbs
V[SUBCAT = intrans, AGR=[NUM=sg,PER=1]] -> 'komme'
V[SUBCAT = intrans, AGR=[NUM=sg,PER=2]] -> 'kommst'
V[SUBCAT = intrans, AGR=[NUM=sg,PER=3]] -> 'kommt'
V[SUBCAT = intrans, AGR=[NUM=pl, PER=1]] -> 'kommen'
V[SUBCAT = intrans, AGR=[NUM=pl, PER=2]] -> 'kommt'
V[SUBCAT = intrans, AGR=[NUM=pl, PER=3]] -> 'kommen'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=sg,PER=1]] -> 'sehe' | 'mag'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=sg,PER=2]] -> 'siehst' | 'magst'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=sg,PER=3]] -> 'sieht' | 'mag'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=sg,PER=1]] -> 'folge' | 'helfe'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=sg,PER=2]] -> 'folgst' | 'hilfst'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=sg,PER=3]] -> 'folgt' | 'hilft'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=pl,PER=1]] -> 'sehen' | 'moegen'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=pl,PER=2]] -> 'sieht' | 'moegt'
V[SUBCAT = trans, OBJCASE=acc, AGR=[NUM=pl,PER=3]] -> 'sehen' | 'moegen'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=pl,PER=1]] -> 'folgen' | 'helfen'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=pl,PER=2]] -> 'folgt' | 'helft'
V[SUBCAT = trans, OBJCASE=dat, AGR=[NUM=pl,PER=3]] -> 'folgen' | 'helfen'
# Adverbs
ADV -> 'Heute'
"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram)

In [49]:
Satz = "Heute sieht der Hund die Katze".split()
for tree in parser.parse(Satz):
    print(tree)

(S[]
  (ADV[] Heute)
  (VP[AGR=[GND='masc', NUM='sg', PER=3]]
    (V[AGR=[NUM='sg', PER=3], OBJCASE='acc', SUBCAT='trans'] sieht)
    (NP[AGR=[GND='masc', NUM='sg', PER=3], CASE='nom']
      (Det[AGR=[GND='masc', NUM='sg', PER=3], CASE='nom'] der)
      (N[AGR=[GND='masc', NUM='sg', PER=3]] Hund))
    (NP[AGR=[GND='fem', NUM='sg', PER=3], CASE='acc']
      (Det[AGR=[GND='fem', NUM='sg', PER=3], CASE='acc'] die)
      (N[AGR=[GND='fem', NUM='sg', PER=3]] Katze))))


In [50]:
# Exercise: 9-12
# Seemingly synonymous verbs have slightly different syntactic properties (Levin, 1993). Consider the
# following patterns of grammaticality for the verbs loaded, filled, and dumped. Can you write grammar
# productions to handle such data?

# a: The farmer loaded the cart with sand
# b: The farmer loaded sand into the cart
# c: The farmer filled the cart with sand
# d: *The farmer filled sand into the cart
# e: *The farmer dumped the cart with sand
# f: The farmer dumped sand into the cart

g = """
% start S
# Grammar Productions
S -> NP VP
NP -> N[Count = False]
NP -> Det N[Count = ?c]
VP -> V[Prep = With] NP 'with' NP
VP -> V[Prep = Into] NP 'into' NP

# Lexical Productions
# Determiner
Det -> 'The' | 'the'
# Nouns
N[Count = True] -> 'farmer' | 'cart'
N[Count = False] -> 'sand'
# Verbs
V[Prep = With] -> 'loaded' | 'filled'
V[Prep = Into] -> 'loaded' | 'dumped'
"""

gram = grammar.FeatureGrammar.fromstring(g)
parser = parse.FeatureEarleyChartParser(gram)

In [51]:
sent = "The farmer loaded the cart with sand".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[] (Det[] The) (N[+Count] farmer))
  (VP[]
    (V[Prep='With'] loaded)
    (NP[] (Det[] the) (N[+Count] cart))
    with
    (NP[] (N[-Count] sand))))


In [52]:
sent = "The farmer loaded sand into the cart".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[] (Det[] The) (N[+Count] farmer))
  (VP[]
    (V[Prep='Into'] loaded)
    (NP[] (N[-Count] sand))
    into
    (NP[] (Det[] the) (N[+Count] cart))))


In [53]:
sent = "The farmer filled the cart with sand".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[] (Det[] The) (N[+Count] farmer))
  (VP[]
    (V[Prep='With'] filled)
    (NP[] (Det[] the) (N[+Count] cart))
    with
    (NP[] (N[-Count] sand))))


In [54]:
sent = "The farmer filled sand into the cart".split()
for tree in parser.parse(sent):
    print(tree)

In [55]:
sent = "The farmer dumped the cart with sand".split()
for tree in parser.parse(sent):
    print(tree)

In [56]:
sent = "The farmer dumped sand into the cart".split()
for tree in parser.parse(sent):
    print(tree)

(S[]
  (NP[] (Det[] The) (N[+Count] farmer))
  (VP[]
    (V[Prep='Into'] dumped)
    (NP[] (N[-Count] sand))
    into
    (NP[] (Det[] the) (N[+Count] cart))))
